#### データソースを準備

In [25]:
from trafilatura import fetch_url, extract

# 遊戯王カードwikiのページを取得
url = 'https://yugioh-wiki.net/index.php?%A1%DA%A5%A2%A5%EB%A5%D0%A5%B9%A4%CE%CD%EE%B0%FD%A1%DB'
filename = 'deck_albaz.txt'
document = fetch_url(url)

# extract()でテキストを抽出
text = extract(document)

# テキストをファイルに保存
if text is not None:
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(text)
else:
    print("No text could be extracted from the document.")

#### チャンク分割

In [26]:
from langchain.document_loaders import TextLoader

# テキストファイルを読み込む
loader = TextLoader(filename, encoding='utf-8')
raw_docs = loader.load()

from langchain.text_splitter import CharacterTextSplitter

# チャンク分割を定義
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=0,
)

# 読み込んだテキストのチャンク分割を実行
docs = text_splitter.split_documents(raw_docs)

print(len(docs))


9


#### embedding

In [27]:
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings

load_dotenv()

# OpenAIのEmbeddingモデルを定義
embeddings = OpenAIEmbeddings(deployment=os.environ["DEPLOYMENT_NAME_EMBEDDINGS"])

from langchain.vectorstores import Chroma

# Chromaベクトルストアを定義
db = Chroma.from_documents(docs, embeddings)

#### 回答

In [28]:
from langchain.chains import  RetrievalQA
from langchain.chat_models import AzureChatOpenAI
import re

retriever = db.as_retriever()

# チャットの組み立て
chat = AzureChatOpenAI(
    deployment_name=os.environ["DEPLOYMENT_NAME"],
    temperature=0
)

# RAGのためのchainを組み立て
rag_qa = RetrievalQA.from_chain_type(
    llm = chat,
    chain_type= "stuff",
    retriever = retriever,
    return_source_documents = True,
)

# 質問文のリストを作成
query_list = ["「アルバスの落胤」デッキのキーカードは？",
              "「アルバスの落胤」の攻は？",
              "「アルバスの落胤」デッキの弱点は？",
              "烙印融合のテキストを教えて",
              "「アルバスの落胤」デッキの派生デッキを3つ教えて",
              "「アルバスの落胤」デッキではリンクモンスターは何が採用される？",
              "「アルバスの落胤」デッキでよく使われるカウンター罠は？",
              "「アルバスの落胤」デッキの戦術は？",
              "「烙印融合」をサーチする方法を教えて",
              "「アルバスの落胤」デッキでは召喚権をどのモンスターに使う？",
              "「アルバスの落胤」デッキにティアラメンツを混ぜるメリットは？",
]

# LLMに渡すチャンクを格納する空リストを作成
context_list = []

# LLMからの回答を格納する空リストを作成
result_list = []

# rag_qaを回して、context_listとresult_listに選ばれたチャンクと回答を格納していく
for query in query_list:
    rag_result = rag_qa({"query": query})

    # contextを取得して、page_contentフィールドをcontext_listに追加に格納
    context = rag_result["source_documents"][0]
    context_list.append(context.page_content)

    # resultを取得して、result_listに格納
    result_list.append(rag_result["result"])

print("チャンクを表示")
print(*context_list, sep='\n++++++++++++\n')
print("*************")
print("回答を表示")
print(*result_list, sep='\n++++++++++++\n')


チャンクを表示
【アルバスの落胤】 †
デッキの概要 †
自身と相手モンスターを素材にフィールド融合を行う《アルバスの落胤》を中心とした【正規融合】デッキ。
《アルバスの落胤》には、烙印を始めとする専用サポートや自身を名指し指定する融合モンスターが多く、それらを軸として構築していく。
- 烙印カードの多くと相性が良い事もあり、その採用枚数も多くなりがちなので、デッキ名は【烙印】と称される事もある。
なお、公式データベースのデッキ検索欄では、「登録カテゴリ」の欄に「アルバスの落胤」と「烙印」の両方が存在する。
《アルバスの落胤/Fallen of Albaz》
効果モンスター
星４/闇属性/ドラゴン族/攻1800/守 0
このカード名の効果は１ターンに１度しか使用できない。
(1)：このカードが召喚・特殊召喚に成功した場合、手札を１枚捨てて発動できる。
融合モンスターカードによって決められた、
このカードを含む融合素材モンスターを自分・相手フィールドから墓地へ送り、
その融合モンスター１体をＥＸデッキから融合召喚する。
この効果で融合召喚する場合、このカード以外の自分フィールドのモンスターを融合素材にできない。
《氷剣竜ミラジェイド/Mirrorjade the Iceblade Dragon》
融合・効果モンスター
星８/闇属性/幻竜族/攻3000/守2500
「アルバスの落胤」＋融合・Ｓ・Ｘ・リンクモンスター
(1)：「氷剣竜ミラジェイド」は自分フィールドに１体しか表側表示で存在できない。
(2)：自分・相手ターンに１度、「アルバスの落胤」を融合素材とする融合モンスター１体をＥＸデッキから墓地へ送って発動できる。
フィールドのモンスター１体を選んで除外する。
++++++++++++
《アルバスの落胤》は自身を融合素材にしなければならないため、効果にチェーンされて除去されると手札コストも含めて手痛い消費となる。
非公開情報の手札誘発や伏せカードによる妨害を極力受けない様に、対策カードを用意した上で効果の発動を行いたい。
手札自体を狙われるのもつらく、ハンデスをされると《アルバスの落胤》の為に残した烙印魔法・罠カードが落とされることになりがち。
相手がモンスターを積極的に出してこないデッキの場合、《アルバスの落胤》や《超融合》が腐りかねないのもネック。
採用するモン